# Klasifikacija konvolucijskim modelom

Prije nego što počnemo degradirat ćemo verziju tensorflowa kako bi izbjegli "cleanup called" poruke koje se pojavljuju na kaggleu kad koristimo tensorflow.

In [ ]:
from IPython.display import clear_output
!pip install -q tensorflow==2.4.1
clear_output()

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

Tensorflow sadrži prigodnu funkcionalnost pomoću koje skinuti s interneta klasične toy skupove podataka za provođenje brzih eksperimenata.
U ovom zadataku koristi ćemo MNIST skup podataka koji se sastoji od slika ručno pisanih znamenki te njihovih točnih labela.

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train','test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
clear_output()

Prije nego što krenemo koristiti skup podataka prvo ga trebamo predprocesuirati.

Slike ćemo predprocesuirati tako da ih normaliziramo.

Kada radimo sa dubokoim modelima ili generalno modelima strojnog učenja u interesu nam je da su značajke koje koristimo na istim skalama jer značajke koje su na većoj skali imaju jači utjecaj na model. Isto tako svaki piksel na slici možemo interpertirati kao jednu značajku te ćemo ga zato normalizirati kako model nebi imao slabiju/jaču preferencu prema njemu.



Već sada ćemo dodati određeni stupanj regularizacije u model kroz razdvajanje skupa podataka na manje grupe. Naime model će bolje raditi ako će procesuirati skup podataka u manjim grupama jer će to smanjivati mogućnost prenaučenosti no isto tako zbog ograničenja radne memorije nije uvijek moguće procesuirati cijeli skup podataka odjednom pogotovo kad je riječ o slikama.

Iskoristi ćemo funkcije cache() i prefetch() kako bi se pobrinuli da grafička kartica tijekom treniranja nema praznog hoda.

In [ ]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255., label

BATCH_SIZE = 32


ds_train = (ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .shuffle(ds_info.splits['train'].num_examples)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE))

ds_test = (ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE))

In [ ]:
LEARNING_RATE = 1e-03
NUM_EPOCHS = 10
AUTO = tf.data.experimental.AUTOTUNE



Za funkciju pogreške uzet ćemo kategoričku unakrsnu entropiju koja je klasična mjera za zadatak višeklasne klasifikacije.

Kako bi izbjegli pretvaranje labela u one hot oblik i time povećali dimenzionalnost skupa podataka koristit ćemo rijetke mjere koje rade sa cijelobrojnim jednodimenzionalnim oznakama.

Tensorflow nudi implementacije rijetke kategoričke unakrsne entropije i rijetke kategoričke točnosti
no ako želimo ostale metrike moramo ih sami implementirati.

Stoga ćemo implementirati metriku za rijetku matricu zabune i iz nje možemo izračunati sve ostale metrike poput preiciznosti,F-mjere i odziva.





In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy,Precision,Recall
import tensorflow.keras.backend as K
import numpy as np


def tp_opr(true,pred):
    return K.sum(true * pred)

def fp_opr(true,pred):
    return K.sum(pred * (1 - true))

def tn_opr(true,pred):
    return K.sum((1-true) * (1 - pred))

def fn_opr(true,pred):
    return K.sum((1-pred) * true)

opr_list = [tp_opr,fp_opr,tn_opr,fn_opr]

def get_class(true, pred, index):
    
    true = K.cast(true,"int32")
    true_oh = tf.one_hot(true,10)
    true_oh = tf.squeeze(true_oh,1)
    
    true = true_oh[:,index]
    
    pred = pred[:,index]
    pred = K.cast(K.equal(pred,K.max(pred)), K.floatx())
    
    return true, pred

def Ti_idx(class_index,opr):
    def TP(true, pred):
        true,pred = get_class(true,pred,class_index)
        return opr(true,pred)
    TP.__name__ = opr.__name__ + f"({class_index})"
    return TP

def get_metrics(classes):
    metrics=[SparseCategoricalAccuracy()]
    if classes:
        for class_i in classes:
            for opr in opr_list:
                metrics.append(Ti_idx(class_i,opr))
    return metrics
metrics = get_metrics([i for i in range(10)])



Sada možemo krenuti sa izgradnjom modela.

Napravit ćemo dva modela. Jedan model biti će relativno jednostavna konvolucijska mreža dok će drugi bit iznimno jednostavna mreža sa samo dva potpuno povezana sloja.
Konvolucijske modele obično koristimo za procesuiranje slika zbog njihove mogućnosti da preko jezgra modelira lokalne interakcije između značajki podataka.

Drugu mrežu koristi ćemo kao mjeru kompleksnosti MNIST-a i time vidjet koliko nam je MNIST zapravo pouzdan skup podataka kada se bavimo eksperimentima klasifikacije.



In [ ]:


def build_model():
    input_layer = layers.Input(shape = (28,28,1))
    conv1 = layers.Conv2D(filters = 16, kernel_size = (3,3),activation = "relu" )(input_layer)
    maxpool1 = layers.MaxPool2D(pool_size = (2,2))(conv1)
    flat = layers.Flatten()(maxpool1)
    inter_layer = layers.Dense(32,activation = "relu")(flat)
    output = layers.Dense(10,activation = "softmax")(inter_layer)
    model = tf.keras.Model(inputs = input_layer, outputs = output)
    optimizer = tf.keras.optimizers.Adam(learning_rate = LEARNING_RATE)
    model.compile(optimizer = optimizer , loss = SparseCategoricalCrossentropy(), metrics = metrics)
    return model

def build_simple_model():
    input_layer = layers.Input(shape = (28,28,1))
    x = layers.Flatten()(input_layer)
    x = layers.Dense(10,activation = "relu")(x)
    output_layer = layers.Dense(10,activation = "softmax")(x)
    model = tf.keras.Model(inputs = input_layer, outputs = output_layer)
    optimizer = tf.keras.optimizers.Adam(learning_rate = LEARNING_RATE)
    model.compile(optimizer = optimizer , loss = SparseCategoricalCrossentropy(), metrics = metrics)
    return model

model = build_model()

simple_model = build_simple_model()






In [ ]:
NUM_EPOCHS = 10
train_history = model.fit(
            ds_train,
            validation_data=ds_test,
            epochs=NUM_EPOCHS, 
            verbose = 0,
        )

In [ ]:
train_history_simp = simple_model.fit(
            ds_train,
            validation_data=ds_test,
            epochs=NUM_EPOCHS, 
            verbose = 0,
        )

Nakon što je treniranje modela završilo kroz History objekte možemo uzeti izračunate metrike i iz njih izgraditi matrice konfuzije koje ćemo onda mikro-uprosječiti.

Preko matrice konfuzije možemo računati mjere preciznosti, odziva i F-mjere koji će nam malo detaljnije
opisati kvalitetu modela.


In [ ]:
import matplotlib.pyplot as plt



def get_conf_matrices(history):
    conf_matrices = list()
    for i in range(10):
        conf_matrices_epoch = list()
        
        tps = history.history[f"val_tp_opr({i})"]
        fps = history.history[f"val_fp_opr({i})"]
        fns = history.history[f"val_fn_opr({i})"]
        tns = history.history[f"val_tn_opr({i})"]
        for j in range(len(tps)):
            conf_mat = np.zeros(shape=(2,2))
            conf_mat[0,0] = tps[j] 
            conf_mat[0,1] = fps[j]
            conf_mat[1,0] = fns[j]
            conf_mat[1,1] = tns[j]
            conf_matrices_epoch.append(conf_mat)
            
        conf_matrices.append(np.array(conf_matrices_epoch))
    return np.array(conf_matrices)

def micro_m(conf_matrices,precision = None,recall = None,F_score = None):
    micro_mat = conf_matrices.sum(axis=0)
    precisions = list()
    recalls = list()
    f_measures = list()
    for i in range(NUM_EPOCHS):
        
        tp = micro_mat[i][0,0]
        fp = micro_mat[i][0,1]
        fn = micro_mat[i][1,0]
        tn = micro_mat[i][1,1]

        out = list()
        if precision:
            precisions.append(tp/(tp+fp))
        if recall:
            recalls.append(tp/(tp+fn))
        if F_score:
            exp = (1+F_score**2)
            f1 = exp * tp
            f2 = f1 + (F_score**2) * fn + fp
            f_measures.append(f1/f2)
        
    if precision:
        out.append(precisions)
    if recall:
        out.append(recalls)
    if F_score:
        out.append(f_measures)
    return tuple(out)
beta = 1


conf_matrices_conv = get_conf_matrices(train_history)
conf_matrices_dense = get_conf_matrices(train_history_simp)


precisions_conv,recalls_conv,f_measures_conv = micro_m(conf_matrices_conv,True,True,beta)
precisions_dense,recalls_dense,f_measures_dense = micro_m(conf_matrices_dense,True,True,beta)






In [ ]:
key_name = "sparse_categorical_accuracy"

plt.plot(train_history_simp.history["val_"+key_name])
plt.plot(train_history.history["val_"+key_name])
plt.plot(train_history_simp.history[key_name])
plt.plot(train_history.history[key_name])
plt.legend(("val_dense","val_conv","train_dense","train_conv"))
plt.xlabel("epoch")
plt.ylabel(key_name)
plt.show()


In [ ]:
key_name = "loss"

plt.plot(train_history_simp.history["val_"+key_name])
plt.plot(train_history.history["val_"+key_name])
plt.plot(train_history_simp.history[key_name])
plt.plot(train_history.history[key_name])
plt.legend(("val_dense","val_conv","train_dense","train_conv"))
plt.xlabel("epoch")
plt.ylabel(key_name)
plt.show()


In [ ]:
plt.xlabel("epoch")
plt.ylabel("precision")
plt.plot(precisions_conv)
plt.plot(precisions_dense)
plt.legend(("conv","dense"))

plt.show()


In [ ]:

plt.plot(recalls_conv)
plt.plot(recalls_dense)
plt.legend(("conv","dense"))
plt.xlabel("epoch")
plt.ylabel("recall")
plt.show()




In [ ]:
plt.plot(f_measures_conv)
plt.plot(f_measures_dense)
plt.legend(("conv","dense"))
plt.xlabel("epoch")
plt.ylabel(f"F{beta}-measure")
plt.show()



Kada bi se fokusirali samo na točnost i preciznost modela mogli bi pomisliti da model radi nedvojbeno dobro no kada malo bolje pogledamo F mjeru i odziv vidimo da i nije baš idealan. Potrebno je napraviti određena poboljšanja isprobavanjem arhitektura i hiperparametara kako bi se došlo do balansiranije F mjere i time do kvalitetnijeg modela.


Isto tako možemo primjetiti da konvolucijski model po metrikama nije daleko od jednostavnog potpuno povezanog modela što nam ukazuje da sam zadatak klasifikacije MNIST skupa i nije najzahtjevniji stoga bi eksperimente trebalo provest i na ostalim skupovima podataka.


Samu evalucija mogli smo proširiti sa dodatnim metrikama poput ispadanja i specifičnosti. Kod samih grafičkih prikaza dodatno su se mogle koristiti PR i ROC krivulje. S obzirom da je riječ isto tako o višeklasnom problemu također smo mogli analizirati sve mjere preko makro uprosječivanja.

Osim samih metrika da bi još više bili sigurni u podatke nad kojima validiramo model također se može otići korak dalje i napraviti K-fold validacija.






# Metričko ugrađivanje

In [ ]:
%%capture
!pip install --upgrade-strategy=only-if-needed tensorflow_similarity[tensorflow] 

U ovom zadataku ponovo ćemo se baviti nadziranim pristupom no ovaj put mjera kojom ćemo učiti
bit će bazirana na udaljenostima značajki slika koje dobivamo modelom, te ćemo time generirati kvalitetan
prostor značajki u kojem su primjeri istih klasa bliže pozicionirani od primjera drugih klasa.

In [ ]:
def get_x(x,y):
    return x
def get_y(x,y):
    return y

def split_xy(ds):
    x = ds.map(get_x)
    y = ds.map(get_y)
    return x,y

Ponovo skidamo skup podataka kako bi ga dobili u željenom formatu za rad knjižnice tensorflow-similarity

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


Sada definiramo MultiShotSampler koji će se pobrinuti da tijekom treniranja kroz model zajedno prolaze adekvatni primjerci klasa.

Primjerice ako za metriku koristimo trojni gubitak za svaki primjer koji uzmemo moramo uzeti još dva primjera, jedan iz iste klase i drugi iz druge klase stoga će se Sampler pobrinuti da taj korak bude izvršen kako spada.

Tijekom treniranja koristi ćemo samo pola klasa, a na skupu za validaciju koristiti ćemo primjere iz svih klasa. Tako ćemo moći vidjeti kako se model snalazi kad izvlači značajke iz primjera nepoznatih klasa i hoće li ih uspjeti odvojiti u zasebne grupe.

In [ ]:
import tensorflow_similarity as tfsim

CLASSES = [2, 3, 1, 7, 9, 6, 8, 5, 0, 4]
NUM_CLASSES = 5
CLASSES_PER_BATCH = NUM_CLASSES
EXAMPLES_PER_CLASS = 10
STEPS_PER_EPOCH = 1000

sampler = tfsim.samplers.MultiShotMemorySampler(
    x_train,
    y_train,
    classes_per_batch=CLASSES_PER_BATCH,
    examples_per_class_per_batch=EXAMPLES_PER_CLASS,
    class_list=CLASSES[:NUM_CLASSES],
    steps_per_epoch=STEPS_PER_EPOCH,
)

Ponovno kreiramo model no ovaj put na kraju modela nemamo klasifikacijsku glavu sa softmaxom
nego samo jedno dimenzionalno polje koje predstavlja značajke slike jer model više ne klasificira podatke nego uspoređuje sličnosti slika.

In [ ]:
from tensorflow_similarity.layers import MetricEmbedding
from tensorflow_similarity.models import SimilarityModel
from tensorflow_similarity.losses import MultiSimilarityLoss
import tensorflow_similarity as tfsim

def build_emb_model():
    inputs = layers.Input(shape=(28, 28, 1))
    
    
    x = layers.Conv2D(32, 3, activation="relu")(inputs)
    x = layers.Conv2D(32, 3, activation="relu")(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(64, 3, activation="relu")(x)
    x = layers.Conv2D(64, 3, activation="relu")(x)
    
    
    x = layers.Flatten()(x)
    outputs = tfsim.layers.MetricEmbedding(64)(x)
    return tfsim.models.SimilarityModel(inputs, outputs)


model_emb = build_emb_model()

In [ ]:
LEARNING_RATE = 0.000005
loss = tfsim.losses.MultiSimilarityLoss(distance="cosine")
optimizer = tf.keras.optimizers.Adam(learning_rate = LEARNING_RATE)
model_emb.compile(optimizer = optimizer,loss = loss)

In [ ]:
EPOCHS = 10
history = model_emb.fit(sampler, epochs=EPOCHS, validation_data=(x_test, y_test))

In [ ]:
embds = model_emb.predict(x_test)

Za vizualizaciju koristit ćemo projector funkciju koja koristi umap algoritam za sažimanje
dimenzija podataka. Podatke smo isto mogli sažeti preko PCA i t-SNE metoda no knjižnica tensorflow-similarity nudi samo umap algoritam koji je najčešće ima najbolje performanse.

In [ ]:
from tensorflow_similarity.visualization import projector


projector(
    embeddings = embds,
    labels = y_test,
    
)

Možemo primjetiti da model radi relativno dobro grupiranje s obzirom da nije vidio pola klasa tijekom
treniranja.
Rad modela bi nadalje mogli poboljšati augmentacijama i podešavanjem hiperparametara modela.

# Metričko ugrađivanje nenadziranim pristupom

U ovom zadataku napraviti ćemo nenadzirani pristup nad skupom podataka MNIST tako što ćemo
prvo model natrenirati preko nadziranog učenja na drugom toydatasetu cifar10, a zatim koristit
njegov ekstraktor značajki nad MNIST skupom.

Prvo moramo prilagoditi format slika skupa cifar10 za format mreže namijenjene za MNIST.
Slike ćemo smanjiti sa dimenzije (32,32) na dimenzije (28,28) te ćemo ih pretvoriti iz tri kanala u samo jedan.

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train','test'],
    as_supervised=True,
    with_info=True,
)
clear_output()


def preproc_img(image, label):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.resize_with_crop_or_pad(image, 28, 28)
    image = tf.reshape(image,(image.shape[1],image.shape[0]))
    image = tf.cast(image, tf.float32) / 255.
    return image, label




ds_train = (ds_train.map(
    preproc_img, num_parallel_calls=tf.data.AUTOTUNE))
ds_test = (ds_test.map(
    preproc_img, num_parallel_calls=tf.data.AUTOTUNE))



Skup podataka potrebno je pretvoriti u oblik koji radi sa MultiSampler-om.

In [ ]:
import matplotlib.pyplot as plt

x_train,y_train = split_xy(ds_train)
x_test,y_test = split_xy(ds_test)

x_train = tfds.as_numpy(x_train)
y_train = tfds.as_numpy(y_train)
x_test = tfds.as_numpy(x_test)
y_test = tfds.as_numpy(y_test)



x_train = np.array([x for x in x_train])

y_train = np.array([y for y in y_train], dtype = np.int32)
x_test = np.array([x for x in x_test])
y_test = np.array([y for y in y_test], dtype = np.int32)


In [ ]:
import tensorflow_similarity as tfsim

CLASSES = [2, 3, 1, 7, 9, 6, 8, 5, 0, 4]
NUM_CLASSES = 10
CLASSES_PER_BATCH = NUM_CLASSES
EXAMPLES_PER_CLASS = 5
STEPS_PER_EPOCH = 1000


sampler = tfsim.samplers.MultiShotMemorySampler(
    x_train,
    y = y_train,
    classes_per_batch=CLASSES_PER_BATCH,
    examples_per_class_per_batch=EXAMPLES_PER_CLASS,
    class_list=CLASSES[:NUM_CLASSES],
    steps_per_epoch=STEPS_PER_EPOCH
)

In [ ]:
LEARNING_RATE = 0.0005
unsupervised_model = build_emb_model()

loss = tfsim.losses.MultiSimilarityLoss(distance="cosine")
optimizer = tf.keras.optimizers.Adam(learning_rate = LEARNING_RATE)
unsupervised_model.compile(optimizer = optimizer,loss = loss)

In [ ]:
EPOCHS = 20
history = unsupervised_model.fit(sampler, epochs=EPOCHS, validation_data=(x_test, y_test))

Nakon što smo istrenirali mrežu možemo ju iskoristiti nad test skupu MNIST slika.

In [ ]:
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = tf.keras.datasets.mnist.load_data()


In [ ]:
embds = unsupervised_model.predict(x_test_mnist)

In [ ]:
from tensorflow_similarity.visualization import projector


projector(
    embeddings = embds,
    labels = y_test_mnist,
)

Za razliku od prethodnog zadatka u ovom slučaju došlo je do značajnijeg spajanja grupa no svejedno možemo primjetiti da je model sposoban kreirati koherentne sektore klasa te bi se sa daljnim poboljšanjima moglo doći do modela kvalitete iz drugog zadatka,a moguće i boljeg.


U ovom slučaju malo se dovodi u upit je li zapravo model upotpunosti nenadziran s obzirom da je riječ
o modelu koji je nadzirano treniran na jednom zadatku i onda primjenjen na drugom.

Kada bi zaista htjeli "pravi" nenadzirani model točnije kada bi koristili isključivo MNIST skup podataka bez labela mogli bi se poslužiti varijacijskim autoenkoderima.